In [1]:
!pip install google-cloud-storage rdkit joblib tqdm pandas pyarrow gcsfs fastparquet --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.4 MB/s eta 0:00:00


In [3]:
from google.colab import files
uploaded = files.upload()

Saving your-service-account-key.json to your-service-account-key.json


In [4]:
!gcloud auth activate-service-account --key-file=your-service-account-key.json

Activated service account credentials for: [colab-access@kinetic-object-437418-v7.iam.gserviceaccount.com]


In [5]:
!gcloud auth list

                        Credentialed Accounts
ACTIVE  ACCOUNT
*       colab-access@kinetic-object-437418-v7.iam.gserviceaccount.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [6]:
!gcloud config set project <project-id>

- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developers console API activation
    url: https://console.developers.google.com/apis/api/cloudresourcemanager.googleapis.com/overview?project=962085889182
- '@type': type.googleapis.com/google.rpc.ErrorInfo
  domain: googleapis.com
  metadata:
    consumer: projects/962085889182
    service: cloudresourcemanager.googleapis.com
  reason: SERVICE_DISABLED
Are you sure you wish to set property [core/project] to 
kinetic-object-437418-v7?

Do you want to continue (Y/n)?  Y

Updated property [core/project].


In [7]:
# !gcloud config list

[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = colab-access@kinetic-object-437418-v7.iam.gserviceaccount.com
project = kinetic-object-437418-v7

Your active configuration is: [default]


In [8]:
# !gcloud auth login --no-launch-browser

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=0hMNU7Z3usQEKjPdZBtNuztU8dbtJn&prompt=consent&token_usage=remote&access_type=offline&code_challenge=IEZHLK1_0trQO8_Es3OnX_zfFZK8nmALabOH-6LTtmo&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AeanS0b2DT8UVTs176mG4_XbVx88J9gLkdJUEomqrS_CzU9qmZj0YdE5Bg9m2TtsLiteYQ

You are now logged in as [rameshwarianand170500@gmail.com].
Your curren

In [9]:
# !gcloud auth list

                        Credentialed Accounts
ACTIVE  ACCOUNT
        colab-access@kinetic-object-437418-v7.iam.gserviceaccount.com
*       rameshwarianand170500@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [10]:
!gcloud storage buckets list

---
creation_time: 2024-11-10T21:37:50+0000
default_storage_class: STANDARD
generation: 1731274669807620758
location: US-EAST1
location_type: region
metageneration: 3
name: drug_prediction
public_access_prevention: enforced
soft_delete_policy:
  effectiveTime: '2024-11-10T21:37:50.427000+00:00'
  retentionDurationSeconds: '604800'
storage_url: gs://drug_prediction/
uniform_bucket_level_access: true
update_time: 2024-12-06T08:27:33+0000


In [11]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
import seaborn as sns
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from tqdm import tqdm
import fastparquet
import pickle
from google.cloud import storage
import os
from sklearn.preprocessing import OneHotEncoder

In [12]:
# Specify the GCS path
gcs_path = 'gs://drug_prediction/train.parquet'

# Enter path of service account
service_account_key_file = '/content/your-service-account-key.json'

In [13]:
# Read the Parquet file into a Pandas DataFrame
train_df = pd.read_parquet(gcs_path, engine='pyarrow', storage_options={"token": service_account_key_file})

# Display the header of the DataFrame
print(train_df.head())

   id                            buildingblock1_smiles buildingblock2_smiles  \
0   0  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
1   1  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
2   2  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
3   3  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   
4   4  C#CC[C@@H](CC(=O)O)NC(=O)OCC1c2ccccc2-c2ccccc21  C#CCOc1ccc(CN)cc1.Cl   

     buildingblock3_smiles                                    molecule_smiles  \
0  Br.Br.NCC1CCCN1c1cccnn1  C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...   
1  Br.Br.NCC1CCCN1c1cccnn1  C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...   
2  Br.Br.NCC1CCCN1c1cccnn1  C#CCOc1ccc(CNc2nc(NCC3CCCN3c3cccnn3)nc(N[C@@H]...   
3        Br.NCc1cccc(Br)n1  C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...   
4        Br.NCc1cccc(Br)n1  C#CCOc1ccc(CNc2nc(NCc3cccc(Br)n3)nc(N[C@@H](CC...   

  protein_name  binds  
0       

In [14]:
# Specify the GCS path
gcs_path = 'gs://drug_prediction/test.parquet'

# Read the Parquet file into a Pandas DataFrame
test_df = pd.read_parquet(gcs_path, engine='pyarrow', storage_options={"token": service_account_key_file})

# Display the header of the DataFrame
print(test_df.head())

          id                            buildingblock1_smiles  \
0  295246830  C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O   
1  295246831  C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O   
2  295246832  C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O   
3  295246833  C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O   
4  295246834  C#CCCC[C@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O   

  buildingblock2_smiles   buildingblock3_smiles  \
0        C=Cc1ccc(N)cc1          C=Cc1ccc(N)cc1   
1        C=Cc1ccc(N)cc1          C=Cc1ccc(N)cc1   
2        C=Cc1ccc(N)cc1          C=Cc1ccc(N)cc1   
3        C=Cc1ccc(N)cc1  CC(O)Cn1cnc2c(N)ncnc21   
4        C=Cc1ccc(N)cc1  CC(O)Cn1cnc2c(N)ncnc21   

                                     molecule_smiles protein_name  
0  C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...         BRD4  
1  C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...          HSA  
2  C#CCCC[C@H](Nc1nc(Nc2ccc(C=C)cc2)nc(Nc2ccc(C=C...          sEH  
3  C#CCCC[C@H](Nc1nc(Nc2ccc(C=

In [17]:
train_df_0 = train_df[train_df['binds'] == 0]
train_df_1 = train_df[train_df['binds'] == 1]

In [19]:
#293M 0's in train data
train_df_0.shape

(293656924, 7)

In [20]:
# 1.5M 1's in train data
train_df_1.shape

(1589906, 7)

In [21]:
# Pivot the dataframe
train_df_wide = train_df.pivot_table(index=["buildingblock1_smiles", "buildingblock2_smiles",
                                "buildingblock3_smiles", "molecule_smiles"],
                         columns="protein_name", values="binds", fill_value=0).reset_index()
# Ensure integer data type for protein columns
train_df_wide = train_df_wide.astype({col: 'int32' for col in train_df_wide.columns if col in ["BRD4", "HSA", "sEH"]})

train_df_wide.head()

all_zeros_train_df = train_df_wide[(train_df_wide['BRD4'] == 0) & (train_df_wide['HSA'] == 0) & (train_df_wide['sEH'] == 0)]

all_ones_train_df = train_df_wide[(train_df_wide['BRD4'] == 1) | (train_df_wide['HSA'] == 1)  | (train_df_wide['sEH'] == 1)]

# Taking twice the total number of ones to introduce bias in the dataset

all_zeros_subset_df = all_zeros_train_df.sample(n= 2 * all_ones_train_df.shape[0], random_state=42)

# Split data for training data which is about 75% of the 0's and 1's
train_zeros = all_zeros_subset_df.sample(frac=0.75, random_state=42)
train_ones = all_ones_train_df.sample(frac=0.75, random_state=42)

# Split data for evaluation data which is about 25% of the 0's and 1's
eval_zeros = all_zeros_subset_df.drop(train_zeros.index)
eval_ones = all_ones_train_df.drop(train_ones.index)

train_df = pd.concat([train_zeros, train_ones])
eval_df = pd.concat([eval_zeros, eval_ones])

In [23]:
local_file_path = "/content/train_df_raw_wide.parquet"
train_df.to_parquet(local_file_path, engine="pyarrow")

In [25]:
# Explicitly set the GOOGLE_APPLICATION_CREDENTIALS environment variable
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/your-service-account-key.json"  # Replace with the path to your key file

# Initialize the GCS client
client = storage.Client()
bucket_name = "drug_prediction"
blob_name = "train_df_raw_wide.parquet"

bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(local_file_path)

print(f"File uploaded to gs://{bucket_name}/{blob_name}")


File uploaded to gs://drug_prediction/train_df_raw_wide.parquet


In [26]:
local_file_path = "/content/eval_df_raw_wide.parquet"
eval_df.to_parquet(local_file_path, engine="pyarrow")

In [27]:
# Initialize the GCS client
client = storage.Client()
bucket_name = "drug_prediction"
blob_name = "eval_df_raw_wide.parquet"

bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(local_file_path)

print(f"File uploaded to gs://{bucket_name}/{blob_name}")

File uploaded to gs://drug_prediction/eval_df_raw_wide.parquet


In [28]:
train_df.head()

protein_name,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,BRD4,HSA,sEH
32775278,O=C(N[C@@H](Cc1ccc(F)cc1)C(=O)O)OCC1c2ccccc2-c...,COc1cccc(-c2cc(N)on2)c1,Cl.Cl.NCCC(=O)Nc1ccncc1,COc1cccc(-c2cc(Nc3nc(NCCC(=O)Nc4ccncc4)nc(N[C@...,0,0,0
10801844,COc1cc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)c...,NCc1cccc(C(F)(F)F)n1,Cl.Cl.NCc1nccc2[nH]ccc12,COc1cc(Nc2nc(NCc3cccc(C(F)(F)F)n3)nc(NCc3nccc4...,0,0,0
63918598,O=C(Nc1ccc(C(=O)O)cc1O)OCC1c2ccccc2-c2ccccc21,C=C(C)C(=O)NCCN.Cl,CSc1nnc(CN)s1,C=C(C)C(=O)NCCNc1nc(NCc2nnc(SC)s2)nc(Nc2ccc(C(...,0,0,0
98024366,O=C(O)[C@H]1Cc2ccccc2CN1C(=O)OCC1c2ccccc2-c2cc...,Nc1cccnc1[N+](=O)[O-],CC1CC(CN)C(C)O1,CC1CC(CNc2nc(Nc3cccnc3[N+](=O)[O-])nc(N3Cc4ccc...,0,0,0
30188343,O=C(N[C@@H](Cc1c(F)c(F)c(F)c(F)c1F)C(=O)O)OCC1...,CN1C(=O)Cc2cc(CN)ccc21.Cl,Cl.Cl.NCc1ccc2scnc2c1,CN1C(=O)Cc2cc(CNc3nc(NCc4ccc5scnc5c4)nc(N[C@@H...,0,0,0


In [29]:
# Define the function for parallel processing
def smiles_to_mol(smiles):
    return Chem.MolFromSmiles(smiles)

# Use joblib's Parallel to convert SMILES in parallel
train_df['molecule'] = Parallel(n_jobs=-1)(delayed(smiles_to_mol)(smiles) for smiles in tqdm(train_df['molecule_smiles']))


100%|██████████| 3397002/3397002 [08:04<00:00, 7006.72it/s] 


In [30]:
# Define the fingerprint generation function
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius=radius, nBits=bits))

# Use Parallel with joblib to process fingerprints in parallel
train_df['ecfp'] = Parallel(n_jobs=-1)(delayed(generate_ecfp)(mol) for mol in tqdm(train_df['molecule']))

train_df_wide = train_df.drop('molecule', axis=1)

train_local_file_path = "/content/train_df_processed_wide.parquet"
train_df_wide.to_parquet(train_local_file_path, engine="pyarrow")

100%|██████████| 3397002/3397002 [37:39<00:00, 1503.17it/s]


In [31]:
# Initialize the GCS client
client = storage.Client()
bucket_name = "drug_prediction"
blob_name = "train_df_wide.parquet"

bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(train_local_file_path)

print(f"File uploaded to gs://{bucket_name}/{blob_name}")

File uploaded to gs://drug_prediction/train_df_wide.parquet


In [32]:
# Use joblib's Parallel to convert SMILES in parallel
eval_df['molecule'] = Parallel(n_jobs=-1)(delayed(smiles_to_mol)(smiles) for smiles in tqdm(eval_df['molecule_smiles']))

100%|██████████| 1132335/1132335 [04:08<00:00, 4565.72it/s] 


In [33]:
# Use Parallel with joblib to process fingerprints in parallel
eval_df['ecfp'] = Parallel(n_jobs=-1)(delayed(generate_ecfp)(mol) for mol in tqdm(eval_df['molecule']))

eval_df_wide = eval_df.drop('molecule', axis=1)


eval_local_file_path = "/content/eval_df_processed_wide.parquet"
eval_df_wide.to_parquet(eval_local_file_path, engine="pyarrow")

100%|██████████| 1132335/1132335 [15:39<00:00, 1205.69it/s]


In [34]:
bucket_name = "drug_prediction"
blob_name = "eval_df_wide.parquet"

bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(eval_local_file_path)

print(f"File uploaded to gs://{bucket_name}/{blob_name}")

File uploaded to gs://drug_prediction/eval_df_wide.parquet


In [35]:
train_df_long = pd.melt(
    train_df_wide,
    id_vars=["buildingblock1_smiles", "buildingblock2_smiles", "buildingblock3_smiles", "molecule_smiles", "ecfp"],
    value_vars=["BRD4", "HSA", "sEH"],
    var_name="protein_names",
    value_name="binds"
)

In [36]:
train_df_long.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,ecfp,protein_names,binds
0,O=C(N[C@@H](Cc1ccc(F)cc1)C(=O)O)OCC1c2ccccc2-c...,COc1cccc(-c2cc(N)on2)c1,Cl.Cl.NCCC(=O)Nc1ccncc1,COc1cccc(-c2cc(Nc3nc(NCCC(=O)Nc4ccncc4)nc(N[C@...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
1,COc1cc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)c...,NCc1cccc(C(F)(F)F)n1,Cl.Cl.NCc1nccc2[nH]ccc12,COc1cc(Nc2nc(NCc3cccc(C(F)(F)F)n3)nc(NCc3nccc4...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
2,O=C(Nc1ccc(C(=O)O)cc1O)OCC1c2ccccc2-c2ccccc21,C=C(C)C(=O)NCCN.Cl,CSc1nnc(CN)s1,C=C(C)C(=O)NCCNc1nc(NCc2nnc(SC)s2)nc(Nc2ccc(C(...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
3,O=C(O)[C@H]1Cc2ccccc2CN1C(=O)OCC1c2ccccc2-c2cc...,Nc1cccnc1[N+](=O)[O-],CC1CC(CN)C(C)O1,CC1CC(CNc2nc(Nc3cccnc3[N+](=O)[O-])nc(N3Cc4ccc...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
4,O=C(N[C@@H](Cc1c(F)c(F)c(F)c(F)c1F)C(=O)O)OCC1...,CN1C(=O)Cc2cc(CN)ccc21.Cl,Cl.Cl.NCc1ccc2scnc2c1,CN1C(=O)Cc2cc(CNc3nc(NCc4ccc5scnc5c4)nc(N[C@@H...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0


In [37]:
train_local_file_path = "/content/train_df_processed.parquet"
train_df_long.to_parquet(train_local_file_path, engine="pyarrow")

In [38]:
bucket_name = "drug_prediction"
blob_name = "processed_train_df.parquet"

bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(train_local_file_path)

print(f"File uploaded to gs://{bucket_name}/artifacts/{blob_name}")

File uploaded to gs://drug_prediction/artifacts/processed_train_df.parquet


In [39]:
eval_df_long = pd.melt(
    eval_df_wide,
    id_vars=["buildingblock1_smiles", "buildingblock2_smiles", "buildingblock3_smiles", "molecule_smiles", "ecfp"],
    value_vars=["BRD4", "HSA", "sEH"],
    var_name="protein_names",
    value_name="binds"
)

In [40]:
eval_local_file_path = "/content/eval_df_processed.parquet"
eval_df_long.to_parquet(eval_local_file_path, engine="pyarrow")

In [41]:
bucket_name = "drug_prediction"
blob_name = "processed_eval_df.parquet"

bucket = client.bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(eval_local_file_path)

print(f"File uploaded to gs://{bucket_name}/artifacts/{blob_name}")

File uploaded to gs://drug_prediction/artifacts/processed_eval_df.parquet


In [43]:
# OneHotEncode the 'protein_name' column
onehot_encoder = OneHotEncoder(sparse_output=False)  # Use sparse=False for dense matrix
onehot_encoder

OneHotEncoder(sparse_output=False)

In [44]:
train_df_long.head()

,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,ecfp,protein_names,binds
0,O=C(N[C@@H](Cc1ccc(F)cc1)C(=O)O)OCC1c2ccccc2-c...,COc1cccc(-c2cc(N)on2)c1,Cl.Cl.NCCC(=O)Nc1ccncc1,COc1cccc(-c2cc(Nc3nc(NCCC(=O)Nc4ccncc4)nc(N[C@...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
1,COc1cc(NC(=O)OCC2c3ccccc3-c3ccccc32)c(C(=O)O)c...,NCc1cccc(C(F)(F)F)n1,Cl.Cl.NCc1nccc2[nH]ccc12,COc1cc(Nc2nc(NCc3cccc(C(F)(F)F)n3)nc(NCc3nccc4...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
2,O=C(Nc1ccc(C(=O)O)cc1O)OCC1c2ccccc2-c2ccccc21,C=C(C)C(=O)NCCN.Cl,CSc1nnc(CN)s1,C=C(C)C(=O)NCCNc1nc(NCc2nnc(SC)s2)nc(Nc2ccc(C(...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
3,O=C(O)[C@H]1Cc2ccccc2CN1C(=O)OCC1c2ccccc2-c2cc...,Nc1cccnc1[N+](=O)[O-],CC1CC(CN)C(C)O1,CC1CC(CNc2nc(Nc3cccnc3[N+](=O)[O-])nc(N3Cc4ccc...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0
4,O=C(N[C@@H](Cc1c(F)c(F)c(F)c(F)c1F)C(=O)O)OCC1...,CN1C(=O)Cc2cc(CN)ccc21.Cl,Cl.Cl.NCc1ccc2scnc2c1,CN1C(=O)Cc2cc(CNc3nc(NCc4ccc5scnc5c4)nc(N[C@@H...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",BRD4,0


In [45]:
protein_onehot_train = onehot_encoder.fit_transform(train_df_long['protein_names'].values.reshape(-1, 1))

In [46]:
protein_onehot_eval = onehot_encoder.transform(eval_df_long['protein_names'].values.reshape(-1, 1))

In [47]:
# Convert lists to NumPy arrays
ecfp_array = np.array(train_df_long['ecfp'].tolist(), dtype=np.float32)
protein_array = np.array(protein_onehot_train.tolist(), dtype=np.float32)
binds_array = np.array(train_df_long['binds'].tolist(), dtype=np.int8)

In [51]:
# Combine arrays
X_train = np.hstack((ecfp_array, protein_array))
y_train = binds_array

In [53]:
print("X:", X_train)
print("y:", y_train)

X: [[0. 1. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 0. 1.]]
y: [0 0 0 ... 1 1 0]


In [ ]:
# Local save directory
save_directory = '/content/processed_data/'
os.makedirs(save_directory, exist_ok=True)

# File paths
x_local_path = os.path.join(save_directory, 'X_train.pkl')
y_local_path = os.path.join(save_directory, 'y_train.pkl')

# Define a function to save a single file
def save_file(file_path, data):
    """Saves data to the specified file path."""
    with open(file_path, 'wb') as f:
        pickle.dump(data, f)
    print(f"File saved to: {file_path}")

# Data to save
files_to_save = [
    (x_local_path, X_train),
    (y_local_path, y_train)
]

# Parallelize file saving
Parallel(n_jobs=-1)(delayed(save_file)(file_path, data) for file_path, data in files_to_save)

print(f"All files saved locally to: {save_directory}")


In [ ]:
x_gcs_path = "artifacts/X.pkl"
y_gcs_path = "artifacts/y.pkl"

bucket = client.bucket(bucket_name)

# Upload X.pkl to GCS
blob = bucket.blob(x_gcs_path)
blob.upload_from_filename(x_local_path)
print(f"File X.pkl uploaded to gs://{bucket_name}/{x_gcs_path}")

# Upload y.pkl to GCS
blob = bucket.blob(y_gcs_path)
blob.upload_from_filename(y_local_path)
print(f"File y.pkl uploaded to gs://{bucket_name}/{y_gcs_path}")

In [ ]:
# Convert lists to NumPy arrays
ecfp_array = np.array(eval_df_long['ecfp'].tolist(), dtype=np.float32)
protein_array = np.array(protein_onehot_eval.tolist(), dtype=np.float32)
binds_array = np.array(eval_df_long['binds'].tolist(), dtype=np.int8)

# Combine arrays
X_eval = np.hstack((ecfp_array, protein_array))
y_eval = binds_array

print("X:", X_eval)
print("y:", y_eval)

In [ ]:
# Local save directory
save_directory = '/content/processed_data/'
os.makedirs(save_directory, exist_ok=True)

# File paths
x_local_path = os.path.join(save_directory, 'X_eval.pkl')
y_local_path = os.path.join(save_directory, 'y_eval.pkl')

# Save files locally
with open(x_local_path, 'wb') as f:
    pickle.dump(X_eval, f)

with open(y_local_path, 'wb') as f:
    pickle.dump(y_eval, f)

print(f"Files saved locally to: {save_directory}")

In [ ]:
x_gcs_path = "artifacts/X_eval.pkl"
y_gcs_path = "artifacts/y_eval.pkl"

# Initialize GCS client
client = storage.Client()
bucket = client.bucket(bucket_name)

# Upload X.pkl to GCS
blob = bucket.blob(x_gcs_path)
blob.upload_from_filename(x_local_path)
print(f"File X_eval.pkl uploaded to gs://{bucket_name}/{x_gcs_path}")

# Upload y.pkl to GCS
blob = bucket.blob(y_gcs_path)
blob.upload_from_filename(y_local_path)
print(f"File y_eval.pkl uploaded to gs://{bucket_name}/{y_gcs_path}")